# Stream Gage Analysis - Identifying Tidally Influenced Gages

In [1]:
from dataretrieval import nwis
#from IPython.display import display
#from datetime import date, timedelta
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#import pytide
#import seaborn as sns
import numpy as np
from ttide.t_tide import t_tide
import csv

In [2]:
def get_data(siteIDs, parameterCode, start_yr, end_yr, startDate='-01-01', endDate = '-12-31'):
    results = []
    for i in range(start_yr,(int(end_yr)+1)):
        data = nwis.get_iv(sites=siteIDs, parameterCd=parameterCode, start=(str(i)+startDate), end=(str(i)+endDate))
        data[0].drop(data[0].columns[[1]], axis = 1, inplace=True)
        results.append(data[0])
    results = pd.concat(results)
    return results

In [3]:
def gage_ID_from_csv(file_name):
    gages_df = pd.read_csv(file_name)
    #gages_df['end_date'] = pd.to_datetime(gages_df['end_date'], format = '%Y-%m-%d')
    gages_df['end_date'] = pd.to_datetime(gages_df['end_date'], format = '%m/%d/%Y')
    gages = gages_df[gages_df['end_date'].dt.year > start_yr]
    gages['site_no'] = gages['site_no'].astype(int)
    gages['site_no'] = gages['site_no'].apply(lambda x: '{0:0>8}'.format(x))
    site_ids = list(gages['site_no'].unique())
    return gages[['site_no', 'dec_lat_va', 'dec_long_va']], site_ids

In [4]:
def tide_pred(results, gage_loc, year=None, amp_threshold = 0.2):
    if len(results.index.name) == 1:
        if (year is not None):
            if year in tide_subset.index.year:
                tide_analys=tide_subset[tide_subset.index.year==year]
                tide_analys.reset_index(inplace=True)
            else:
                pass
        else:
            tide_analys = tide_subset
            tide_analys.reset_index(inplace=True)
        #startdate = tide_analys['datetime'][0]
        #enddate = startdate + timedelta(days=7)
        stream_data = tide_analys.groupby([pd.Grouper(key='datetime', freq='H'),'site_no']).mean()
        stream_data.reset_index(inplace=True)
        
        stream_data['demean'] = stream_data['00065'] - stream_data['00065'].mean()
        #tfit_e = t_tide(stream_data['demean'].values, out_style='classic')

        tfit_e = t_tide(stream_data['demean'].values, out_style='none')
        if sum([i[0]for i in tfit_3['tidecon']]) > amp_threshold:
            gage_loc['Influence'] = 'Tidal'
        else:
            gage_loc['Influence'] = 'None'


    else:
        for i in results.index.levels[0]:
            tide_subset = results.loc[[i]].copy()
            tide_subset.reset_index(inplace=True)
            tide_subset.set_index('datetime',inplace=True)
            if (year is not None):
                if year in tide_subset.index.year:
                    tide_analys=tide_subset[tide_subset.index.year==year]
                    tide_analys.reset_index(inplace=True)
                else:
                    continue
            else:
                tide_analys = tide_subset
                tide_analys.reset_index(inplace=True)
            #startdate = tide_analys['datetime'][0]
            #enddate = startdate + timedelta(days=7)
            stream_data = tide_analys.groupby([pd.Grouper(key='datetime', freq='H'),'site_no']).mean()
            stream_data.reset_index(inplace=True)
            
            stream_data['demean'] = stream_data['00065'] - stream_data['00065'].mean()
            #tfit_e = t_tide(stream_data['demean'].values, out_style='classic')

            tfit_e = t_tide(stream_data['demean'].values, out_style='none')
            
            if sum([i[0]for i in tfit_3['tidecon']]) > amp_threshold:
                gage_loc['Influence'] = 'Tidal'
            else:
                gage_loc['Influence'] = 'None'

    
        return gage_loc


*Use this cell to detail what user information should be added to the block below*

In [6]:
# Set the parameters needed for the web service call

#parameterCode = '00060'  # Discharge (mean, daily - ft^3/sec)
#parameterCode = '00061'  # Discharge (instantaneous - ft^3/sec)
parameterCode = '00065'   #Gage Heights (ft)
#startDate = '-07-01'
#endDate = '-07-30'
start_yr = 2018
end_yr = 2022

gages, stations = gage_ID_from_csv(r'NJ-2_Gage_Sites.csv')
results=get_data(stations, parameterCode, start_yr, end_yr)
#ttide_results = tide_pred(results, gages) ->testing below currently

display(results.head())
#still need to assign_attrib as tidal vs. non-tidal


/var/folders/1l/2r02jwq11s19l91rj_gktc9m6cb00_/T/ipykernel_8958/2569509585.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gages['site_no'] = gages['site_no'].astype(int)
/var/folders/1l/2r02jwq11s19l91rj_gktc9m6cb00_/T/ipykernel_8958/2569509585.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gages['site_no'] = gages['site_no'].apply(lambda x: '{0:0>8}'.format(x))


00065
site_no  datetime                        
01367715 2018-06-22 17:30:00+00:00   1.75
         2018-06-22 17:45:00+00:00   1.75
         2018-06-22 18:00:00+00:00   1.75
         2018-06-22 18:15:00+00:00   1.75
         2018-06-22 18:30:00+00:00   1.75

In [86]:
###FOR DEMO PURPOSES RUN THIS CODE BLOCK###
#Still cleaning up some of the logic functions in the tide_pred function
#Need to fix loop for running multiple gages in this demo block - only outputting results for one gage currently
#gage_loc = pd.DataFrame(columns=[['site_no', 'Influence']])
gage_loc = []
##gage_loc['site_no']=results.index.levels[0]
##print(gage_loc[['site_no'][0]].dtypes())
##display(gage_loc)

##gage_loc['site_no']=gage_loc['site_no'].astype('string')
##print(type(gage_loc['site_no'][0]))
##rint((gage_loc.dtypes))
#print(gage_loc['site_no'])
display(gage_loc)
for i in results.index.levels[0]:
    #print(type(i))
#for i in results_test.index.levels[0]:
    tide_subset = results.loc[[i]].copy()
    #tide_subset = results_test.loc[[i]].copy()
    tide_subset.reset_index(inplace=True)
    tide_subset.set_index('datetime',inplace=True)


    display(tide_subset.head)
    tide_subset.reset_index(inplace=True)
    tide_subset.set_index('datetime',inplace=True)



    tide_analys = tide_subset
    tide_analys.reset_index(inplace=True)
    stream_data = tide_analys.groupby([pd.Grouper(key='datetime', freq='H'),'site_no']).mean()
    stream_data.reset_index(inplace=True)

    stream_data['demean'] = stream_data['00065'] - stream_data['00065'].mean()
            #tfit_e = t_tide(stream_data['demean'].values, out_style='classic')

    t_fit = t_tide(stream_data['demean'].values, out_style='pandas')

    #print(stream_data['site_no'].unique())
    #gage_loc['site_no'] = stream_data['site_no'].unique()
    amp_threshold = 0.2
    display(gage_loc)
    if sum([i[0]for i in t_fit['tidecon']]) > amp_threshold:
        #print(gage_loc.site_no, 'Tidal')
        #gage_loc['Influence'][gage_loc.site_no == int(i)] = 'Tidal'
        gage_loc.append({'Site_No':i ,'Influence':'Tidal'})
       
    else:
        #gage_loc['Influence'][gage_loc.site_no == int(i)] = 'None'
        #print(gage_loc.site_no, 'Non-Tidal')
        gage_loc.append({'Site_No':i , 'Influence':'Non-Tidal'})
    
gage_loc_df = pd.DataFrame(gage_loc)
display(gage_loc_df)

#Appending Gage information to include lat long


[]

<bound method NDFrame.head of                             site_no  00065
datetime                                  
2018-06-22 17:30:00+00:00  01367715   1.75
2018-06-22 17:45:00+00:00  01367715   1.75
2018-06-22 18:00:00+00:00  01367715   1.75
2018-06-22 18:15:00+00:00  01367715   1.75
2018-06-22 18:30:00+00:00  01367715   1.75
...                             ...    ...
2023-01-01 03:45:00+00:00  01367715   2.44
2023-01-01 04:00:00+00:00  01367715   2.44
2023-01-01 04:15:00+00:00  01367715   2.45
2023-01-01 04:30:00+00:00  01367715   2.46
2023-01-01 04:45:00+00:00  01367715   2.46

[146248 rows x 2 columns]>

Number of observations = 36570
Number of observations used = 36569
Record length (days) = 1523.75
Phases at central time

x0= 0.00864  xtrend= 0
var(data)= 0.26    var(prediction)= 0.03    var(residual)= 0.23
var(prediction)/var(data) (%) = 10.7

            Tidal amplitude and phase with 95 % CI estimates
                 Freq        Amp    Amp Err      Phase  Phase Err        SNR
Tide                                                                        
SA           0.000114       0.23       0.07      152.6       20.6      1e+01
SSA          0.000228       0.06       0.07      248.4       61.1        0.8
MSM          0.001310       0.04       0.06      159.1       98.3        0.5
MM           0.001512       0.03       0.06      209.7      116.6        0.3
MSF          0.002822       0.02       0.05      284.8      187.0       0.09
MF           0.003050       0.01       0.05      328.6      253.9       0.02
ALP1         0.034397       0.00       0.00      212.4       85.7        0.6

[]

<bound method NDFrame.head of                             site_no  00065
datetime                                  
2018-01-01 05:00:00+00:00  01377000   1.66
2018-01-01 05:15:00+00:00  01377000   1.66
2018-01-01 05:30:00+00:00  01377000   1.66
2018-01-01 05:45:00+00:00  01377000   1.66
2018-01-01 06:00:00+00:00  01377000   1.66
...                             ...    ...
2023-01-01 03:45:00+00:00  01377000   1.77
2023-01-01 04:00:00+00:00  01377000   1.77
2023-01-01 04:15:00+00:00  01377000   1.78
2023-01-01 04:30:00+00:00  01377000   1.78
2023-01-01 04:45:00+00:00  01377000   1.78

[175066 rows x 2 columns]>

Number of observations = 43776
Number of observations used = 43775
Record length (days) = 1824.00
Phases at central time

x0= -5.5e-05  xtrend= 0
var(data)= 0.19    var(prediction)= 0.00    var(residual)= 0.19
var(prediction)/var(data) (%) = 1.5

            Tidal amplitude and phase with 95 % CI estimates
                 Freq        Amp    Amp Err      Phase  Phase Err        SNR
Tide                                                                        
SA           0.000114       0.08       0.05      254.7       35.4          2
SSA          0.000228       0.06       0.05      215.8       53.9          2
MSM          0.001310       0.03       0.04      105.2      109.2        0.4
MM           0.001512       0.01       0.04      144.0      165.0        0.1
MSF          0.002822       0.03       0.04      223.5      102.5        0.5
MF           0.003050       0.02       0.04       66.9      122.1        0.4
ALP1         0.034397       0.00       0.00       25.2       58.6          1

[{'Site_No': '01367715', 'Influence': 'Tidal'}]

,Site_No,Influence
0,01367715,Tidal
1,01377000,Tidal


All code below is for dev purposes and may not run with the currnet work flow. 

In [180]:
#FOR PlOTTING RELATIONSHIP BETWEEN GAGE SITES##
##Does not work for individual runs yet - has to be manipulated for each gage of interest (currently set up for Jacksonville, FL)##

harm_analysis_02236125 = pd.read_csv('test_02236125.csv', header = 9, sep='\s{2,}', engine = 'python')
harm_analysis_02246010 = pd.read_csv('test_02246010.csv', header = 9, sep='\s{2,}', engine = 'python')
harm_analysis_02246500 = pd.read_csv('test_02246500.csv', header = 9, sep='\s{2,}', engine = 'python')
harm_analysis_302309081333001 = pd.read_csv('test_302309081333001.csv', header = 9, sep='\s{2,}', engine = 'python')
harm_analysis_2245340 = pd.read_csv('test_02245340.csv', header = 9, sep='\s{2,}', engine = 'python')
#harm_analysis_302309081333001 = pd.read_csv('test_302309081333001.csv', header = 9, sep='\s{2,}', engine = 'python')
#harm_analysis_302309081333001 = pd.read_csv('test_302309081333001.csv', header = 9, sep='\s{2,}', engine = 'python')

analyses = {'harm_analysis_302309081333001':harm_analysis_302309081333001, 'harm_analysis_02246500':harm_analysis_02246500,  'harm_analysis_02246010':harm_analysis_02246010, 'harm_analysis_02236125':harm_analysis_02236125, 'harm_analysis_2245340': harm_analysis_2245340}
#print((analyses))

tide_results_for_scatter = pd.DataFrame()

#display(harm_analysis_02236125.head())
#display(harm_analysis_02246010.head())
#display(harm_analysis_02246500.head())
#display(harm_analysis_302309081333001.head())
#for i in results.index.levels[0]:
for key, val in analyses.items():
    #print(val)
    #print([k for k, j in locals().items() )
    #print(val.loc[['M2','K1','O1'],'Amp'])
    tide_results_for_scatter[key]= val.loc[['M2','K1','O1'],'Amp']
    #tide_results_for_scatter ['']
    #display(tide_results_for_scatter)
riv_mi = pd.read_csv('Demo/gage_info.csv', header =1)
riv_mi.set_index('Site No.', inplace = True)
riv_mi_vals = riv_mi['Riv Mi'][::-1]
#if tide_results_for_scatter.columns. is in (riv_mi['Site No.']):
    #tide_results_for_scatter['riv mi'] = riv_mi['Riv Mi']
#for i in tide_results_for_scatter.columns:
    
#tide_results_for_scatter['Riv Mi'].apply(lambda x: )
tide_results_for_scatter.loc['riv id'] = riv_mi_vals.values

tide_results_for_scatter.reset_index(inplace=True)
pd.melt(tide_results_for_scatter)
display(tide_results_for_scatter)

,Tide,harm_analysis_302309081333001,harm_analysis_02246500,harm_analysis_02246010,harm_analysis_02236125,harm_analysis_2245340
0,M2,1.89,0.86,0.46,0.03,0.43
1,K1,0.29,0.14,0.10,0.01,0.10
2,O1,0.18,0.09,0.08,0.01,0.08
3,riv id,11.00,24.00,61.00,NaN,129.75
